In [1]:
import operator
import numpy as np
import gensim
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
glove_model = KeyedVectors.load_word2vec_format("D:/wiki.de.vec", binary=False)

In [3]:
glove_model.init_sims(replace=True)

In [4]:
def decapitalize(word):
    if len(word) > 1:
        word = word[0].lower() + word[1:]
    return word

In [5]:
def all_dist(vector_1,vector_2):
    cur_dist = 0
    for i in range((len(vector_1))):
        if i == 0 or i == 3:
            try:
                ith_dist = glove_model.similarity(decapitalize(vector_1[i]), decapitalize(vector_2[i]))
                cur_dist = 5*(1-ith_dist)*(1-ith_dist)
            except:
                ith_dist = 0
                cur_dist = 5*(1-ith_dist)*(1-ith_dist)
        else:
            if vector_1[i] != vector_2[i]:
                cur_dist += 1
    return cur_dist

In [6]:
file = open("C:/Users/1/Desktop/thesis/corpus_final/compounds_final.txt", "r", encoding="utf-8")
lines = file.readlines()

In [7]:
matrix = []
for line in lines:
    vectors = []
    types = []
    line = line.strip("\n")
    items = line.split("/")
    for item in items[1:8]:
        vectors.append(item)
    types.append(items[-1])
    tp = (vectors,types)
    matrix.append(tp)

In [8]:
matrixX = []
matrixY = []
for item in matrix:
    matrixX.append(item[0])
    matrixY.append(item[1][0])

In [9]:
X = np.array(matrixX)
y = np.array(matrixY)

In [10]:
kf = KFold(n_splits=3, shuffle=True)

In [11]:
kf.get_n_splits(X)

3

In [12]:
all_predicts = []
tests = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    predictions = []
    for vector in X_test:
        distances = []
        types = []
        preds = []
        for x in range(len(X_train)):
            distance = all_dist(X_train[x], vector)
            distances.append(distance)
            types.append(y_train[x])
        distances = np.array(distances)
        inds = distances.argsort()[:5]
        for ind in inds:
            preds.append(str(types[ind]))
        typ = Counter(preds).most_common(1)[0][0]
        predictions.append(typ)
    y_test = y_test.tolist()
    all_predicts.extend(predictions)
    tests.extend(y_test)
    
print(classification_report(tests, all_predicts))

             precision    recall  f1-score   support

         AN       0.59      0.63      0.61      1961
          N       0.60      0.64      0.62      2124
         NN       0.38      0.23      0.29       664

avg / total       0.57      0.58      0.57      4749

